# Deadlock & Daemon Threads in Python

## Introduction

### Threading in Python:
Threads allow multiple parts of a program to run concurrently within the same process.

#### Deadlock:

- A deadlock occurs when two or more threads wait indefinitely for resources locked by each other.
- Example: Thread-1 holds Lock-A and waits for Lock-B, while Thread-2 holds Lock-B and waits for Lock-A.

#### Daemon Threads:

- Daemon threads run in the background and are killed automatically when the main program exits.
- Used for background tasks like monitoring/logging.
- Contrast: Non-daemon threads keep running until they finish, even if the main program ends.

## Demonstrating Deadlock in Python

In [ ]:
import threading
import time

LA = threading.Lock()
LB = threading.Lock()

def taskX():
    with LA:
        print("Task X acquired Lock A")
        time.sleep(1)
        with LB:
            print("Task X acquired Lock B")

def taskY():
    with LB:
        print("Task Y acquired Lock B")
        time.sleep(1)
        with LA:
            print("Task Y acquired Lock B")

t1 = threading.Thread(target = taskX)
t2 = threading.Thread(target = taskY)

t1.start()
t2.start()

t1.join()
t2.join()

Task X acquired Lock A
Task Y acquired Lock B


- Two locks are created (LA and LB) representing two shared resources.

- taskX()
    - taskX first acquires Lock A.
    - It then waits 1 second before trying to acquire Lock B

- taskX()
    - taskY acquires Lock B first.
    - After 1 second, it tries to acquire Lock A.

### What Happens (Deadlock Scenario)

1) Thread-1 (Task X) acquires Lock A.
2) Thread-2 (Task Y) acquires Lock B.
3) Now:
    - Thread-1 is waiting for Lock B (held by Thread-2).
    - Thread-2 is waiting for Lock A (held by Thread-1).
4) Both threads are stuck forever → Deadlock.

## Avoiding Deadlock (Best Practices)

- Always acquire locks in the same order.
- Use try...acquire(timeout=...) to avoid indefinite waiting.

**1) LA.acquire(timeout=1)**

- The thread tries to acquire LA (Lock A).
- If it cannot acquire it within 1 second, it gives up and continues execution (avoiding infinite waiting).

**2) If LA is acquired**
- The thread holds Lock A.
- It prints a confirmation message.
- 
**3) Wait 1 second**
- Simulates doing some work while holding Lock A.

**4) LB.acquire(timeout=1)**
- While still holding Lock A, it tries to acquire Lock B.
- Again, if it cannot acquire Lock B within 1 second, it gives up instead of blocking forever.

**5) If Lock B is acquired**
- Prints success message.
- Releases Lock B immediately after finishing the work.

**6) Finally releases Lock A**
- Ensures that Lock A is freed before function exits.

In [ ]:
def safe_task():
    if LA.acquire(timeout=1):
        print("Safe Task1 acquired Lock1")
        time.sleep(1)
        if LB.acquire(timeout=1):
            print("Safe Task1 acquired Lock2")
            LB.release()
        LA.release()

#### Why This Prevents Deadlock

In the original deadlock example:
- Task X waits forever for Lock B.
- Task Y waits forever for Lock A.

In this safe version:
- If Task X can’t acquire Lock B within 1 second, it gives up and releases Lock A.
- This lets Task Y eventually acquire Lock A and proceed.

So instead of freezing forever, one thread fails gracefully and the system keeps moving.

## Demonstrating Daemon Threads

In [ ]:
import threading
import time

def background_task():
    while True:
        print("Background task running...")
        time.sleep(1)

# Daemon thread
t = threading.Thread(target=background_task, daemon=True)
t.start()

print("Main thread finished.")

- The daemon thread keeps printing messages in the background.
- But as soon as the main thread ends, the daemon thread also terminates automatically.

### Real-World Applications

- **Deadlock awareness:** Important in multi-threaded programs handling databases, files, or shared resources.
- **Daemon threads:** Useful for background monitoring, logging, or scheduled tasks without blocking program termination.